In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
alt.data_transformers.enable('json')
from sublimpy import tidy, utils
import pytz

In [2]:
seconds_in_timestep = 60*30
from metpy.constants import density_water

# Open datasets

In [3]:
# Open SOS Measurement Dataset
################################################
start_date = '20221130'
end_date = '20230509'
# open files
tidy_df = pd.read_parquet(f'tidy_df_{start_date}_{end_date}_noplanar_fit_clean.parquet')
# convert time column to datetime
tidy_df['time'] = pd.to_datetime(tidy_df['time'])
# limit data to our dates of interest, based on continuous snow cover at Kettle Ponds
tidy_df = tidy_df.set_index('time').sort_index().loc[start_date:end_date].reset_index()

tidy_df = utils.modify_df_timezone(tidy_df, pytz.UTC, 'US/Mountain')


# Open Turbpy Model Ensemble Dataset
################################################
model_df = pd.read_parquet("model_results.parquet")
# add a bunch of columns that are descriptive, from the config column which has multiple bits of info
model_df['z0'] = model_df['config'].apply(
    lambda v: v.split(' ')[-1]
)
model_df['e_sat_curve'] = model_df['config'].apply(
    lambda v: 'metpy' if 'metpy' in v else 'alduchov'
)
model_df['surface_measurement'] = model_df['config'].apply(
    lambda v: v.split(' ')[-3]
)
model_df['scheme'] = model_df['config'].apply(
    lambda v: 'andreas' if 'andreas lengths' in v else 'yang'
)
model_df['most_config'] = model_df['config'].apply(lambda s: ' '.join(s.split(' ')[:-3]))
# remove the scalar roughness length parameterization info 
model_df['most_config'] = model_df['most_config'].str.replace(' andreas lengths', '')
model_df['latent heat flux (mm)'] = -model_df['latent heat flux']*seconds_in_timestep/density_water/2838

model_df = utils.modify_df_timezone(model_df, pytz.UTC, 'US/Mountain')


# Open COARE model results
################################################
coare_model_results = pd.read_parquet("coare_model_results.parquet").reset_index()
coare_model_results['z0'] = coare_model_results.config.str.split(' ').apply(lambda x: x[-2])
coare_model_results['surface_measurement'] = coare_model_results.config.str.split(' ').apply(lambda x: x[0])
coare_model_results['e_sat_curve'] = coare_model_results.config.str.split(' ').apply(lambda x: x[1])
coare_model_results['meas_height'] = coare_model_results.config.str.split(' ').apply(lambda x: int(x[-1].split('m')[0]))

coare_model_results['hlb_mm'] = coare_model_results['hlb_gperm2s']*seconds_in_timestep/density_water

coare_model_results = utils.modify_df_timezone(coare_model_results, pytz.UTC, 'US/Mountain')

In [4]:
tidy_df[tidy_df.variable.isin([
    'w_h2o__1m_c',
    'w_h2o__2m_c',
    'w_h2o__3m_c',
    'w_h2o__5m_c',
    'w_h2o__10m_c',
    'w_h2o__15m_c',
    'w_h2o__20m_c',
])].groupby("variable").value.mean()

variable
w_h2o__10m_c    0.002937
w_h2o__15m_c    0.002159
w_h2o__20m_c    0.002323
w_h2o__2m_c     0.002321
w_h2o__3m_c     0.002756
w_h2o__5m_c     0.002824
Name: value, dtype: float64

# Analysis

In [5]:
tidy_df = tidy.tidy_df_add_variable(
    tidy_df,
    (
        tidy_df.query("variable == 'SF_avg_1m_ue'")['value'].values + 
        tidy_df.query("variable == 'SF_avg_2m_ue'")['value'].values
    ), 
    'SF_avg_ue',
    'snow flux',
    1,
    'ue',
)


In [6]:
coare_model_results.columns

Index(['time', 'config', 'hsb', 'hlb', 'tau', 'zo', 'zot', 'zoq', 'L', 'usr',
       'tsr', 'qsr', 'dter', 'dqer', 'hl_webb', 'Cd', 'Ch', 'Ce', 'Cdn_10',
       'Chn_10', 'Cen_10', 'rr', 'rt', 'rq', 'hlb_gperm2s', 'z0',
       'surface_measurement', 'e_sat_curve', 'meas_height', 'hlb_mm'],
      dtype='object')

In [7]:
coare_model_results.head()

,time,config,hsb,hlb,tau,zo,zot,zoq,L,usr,...,Cen_10,rr,rt,rq,hlb_gperm2s,z0,surface_measurement,e_sat_curve,meas_height,hlb_mm
0,2022-11-29 17:00:00,Tsurf_c e_sat_alduchov z0_andreas 3m,1.986449,9.420090,0.025311,0.000687,0.000110,0.000136,-111.311655,0.163917,...,0.001490,9.255205,1.460589,1.805124,0.003319,z0_andreas,Tsurf_c,e_sat_alduchov,3,0.005975
1,2022-11-29 17:30:00,Tsurf_c e_sat_alduchov z0_andreas 3m,0.609089,17.194120,0.042192,0.000448,0.000091,0.000111,-379.871783,0.210755,...,0.001400,7.775596,1.551745,1.895916,0.006059,z0_andreas,Tsurf_c,e_sat_alduchov,3,0.010906
2,2022-11-29 18:00:00,Tsurf_c e_sat_alduchov z0_andreas 3m,-1.383714,12.786376,0.026696,0.000370,0.000125,0.000149,440.767566,0.167234,...,0.001411,5.106162,1.715603,2.040416,0.004505,z0_andreas,Tsurf_c,e_sat_alduchov,3,0.008110
3,2022-11-29 18:30:00,Tsurf_c e_sat_alduchov z0_andreas 3m,-1.103841,2.673624,0.004362,0.000091,0.000151,0.000195,20.459927,0.067514,...,0.001271,0.511193,0.858169,1.106702,0.000942,z0_andreas,Tsurf_c,e_sat_alduchov,3,0.001696
4,2022-11-29 19:00:00,Tsurf_c e_sat_alduchov z0_andreas 3m,-1.394393,1.225486,0.005249,0.003317,0.000136,0.000178,19.450489,0.074026,...,0.001826,20.694098,0.956043,1.248783,0.000432,z0_andreas,Tsurf_c,e_sat_alduchov,3,0.000777


In [8]:
coare_model_results['psi_m'] = np.log(
    coare_model_results['meas_height'] / coare_model_results['zo']
) + (
    0.4 / coare_model_results['Cd']**0.5
)
coare_model_results['zeta'] = coare_model_results['meas_height'] / coare_model_results['L']

In [9]:
alt.Chart(
    coare_model_results.query(
        "config == 'Tsurf_c e_sat_alduchov z0_andreas 3m'"
    )
).mark_circle().encode(
    alt.X("zeta:Q"),
    alt.Y("psi_m")
)

alt.Chart(...)

# Define times for different categories

In [81]:
bs_times = tidy_df.query("variable == 'SF_avg_ue'").query("value > 0").time
nobs_times = tidy_df.query("variable == 'SF_avg_ue'").query("value == 0").time
decoupled_times = tidy_df.query("variable == 'omega_3m_c'").query("value < 0.43").time
weaklycoupled_times = tidy_df.query("variable == 'omega_3m_c'").query("value >= 0.43").query("value <= 0.61").time
coupled_times = tidy_df.query("variable == 'omega_3m_c'").query("value > 0.61").time

ri_stable_times = tidy_df.query("variable == 'Ri_3m_c'").query("value > 0.25").time
ri_unstable_times = tidy_df.query("variable == 'Ri_3m_c'").query("value < -0.01").time
ri_neutral_times = tidy_df.query("variable == 'Ri_3m_c'").query("value >= -0.01").query("value <= 0.25").time

tgrad_stable_times = tidy_df.query("variable == 'temp_gradient_3m_c'").query("value > 0.01").time
tgrad_unstable_times = tidy_df.query("variable == 'Ri_3m_c'").query("value < -0.01").time
tgrad_neutral_times = tidy_df.query("variable == 'Ri_3m_c'").query("value >= -0.01").query("value <= 0.01").time

In [82]:
lhflux_measurements_df = tidy_df[tidy_df.variable.isin([
    'w_h2o__1m_c',
    'w_h2o__2m_c',
    'w_h2o__3m_c',
    'w_h2o__5m_c',
    'w_h2o__10m_c',
    'w_h2o__15m_c',
    'w_h2o__20m_c',
])]
lhflux_measurements_df

,time,variable,value,height,tower,measurement
223,2022-11-29 17:00:00,w_h2o__2m_c,0.001048,2.0,c,w_h2o_
342,2022-11-29 17:00:00,w_h2o__3m_c,0.001886,3.0,c,w_h2o_
672,2022-11-29 17:00:00,w_h2o__20m_c,0.008915,20.0,c,w_h2o_
676,2022-11-29 17:00:00,w_h2o__10m_c,0.010222,10.0,c,w_h2o_
678,2022-11-29 17:00:00,w_h2o__5m_c,0.003238,5.0,c,w_h2o_
...,...,...,...,...,...,...
5278155,2023-05-09 17:30:00,w_h2o__3m_c,0.003844,3.0,c,w_h2o_
5278169,2023-05-09 17:30:00,w_h2o__5m_c,0.002175,5.0,c,w_h2o_
5278204,2023-05-09 17:30:00,w_h2o__15m_c,NaN,15.0,c,w_h2o_
5278206,2023-05-09 17:30:00,w_h2o__20m_c,NaN,20.0,c,w_h2o_


## Calculate frequency of lh flux divergence using 10% and 30% thresholds

In [100]:
src = lhflux_measurements_df.pivot(index='time', columns='variable', values='value')
src['percent diff'] = (src['w_h2o__3m_c'] - src['w_h2o__5m_c']) / src['w_h2o__5m_c']

In [101]:
src['percent diff'].median(), src['percent diff'].mean(), src['percent diff'].max(), src['percent diff'].min()

(-0.10465147127322266,
 -0.005035193055386951,
 1698.5479166666667,
 -460.8122575640031)

In [111]:
print(len(src)), print(len(src[src["percent diff"] > 0.1])/len(src)), print(len(src[src["percent diff"] > 0.3])/len(src))

7728
0.25556418219461696
0.17650103519668736


(None, None, None)

In [112]:
lhflux_measurements_df['vertical_divergence_percentage'] = lhflux_measurements_df.groupby('time').apply(
    lambda df: np.abs(df['value'] - df['value'].mean()) / df['value'].mean()
).values

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_60974/826112600.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lhflux_measurements_df['vertical_divergence_percentage'] = lhflux_measurements_df.groupby('time').apply(
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_60974/826112600.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lhflux_measurements_df['vertical_divergence_percentage'] = lhflux_measurements_df.groupby('time').appl

In [113]:
(lhflux_measurements_df.groupby("time")['vertical_divergence_percentage'].apply(lambda x: np.abs(x).mean()) > 0.3).value_counts()

vertical_divergence_percentage
True     5793
False    1935
Name: count, dtype: int64

In [114]:
(lhflux_measurements_df.groupby("time")['vertical_divergence_percentage'].apply(lambda x: np.abs(x).mean()) > 0.3).value_counts()

vertical_divergence_percentage
True     5793
False    1935
Name: count, dtype: int64

In [115]:
(lhflux_measurements_df.groupby("time")['vertical_divergence_percentage'].apply(lambda x: np.abs(x).mean()) > 0.1).value_counts()

vertical_divergence_percentage
True     7442
False     286
Name: count, dtype: int64

## Plot mean lh flux vertical profiles for different conditions

In [116]:
bs_lhflux_df = lhflux_measurements_df[lhflux_measurements_df.time.isin(bs_times)]
nobs_lhflux_df = lhflux_measurements_df[lhflux_measurements_df.time.isin(nobs_times)]
decoupled_df = lhflux_measurements_df[lhflux_measurements_df.time.isin(decoupled_times)]
weaklycoupled_df = lhflux_measurements_df[lhflux_measurements_df.time.isin(weaklycoupled_times)]
coupled_df = lhflux_measurements_df[lhflux_measurements_df.time.isin(coupled_times)]

ri_stable_df = lhflux_measurements_df[lhflux_measurements_df.time.isin(ri_stable_times)]
ri_unstable_df = lhflux_measurements_df[lhflux_measurements_df.time.isin(ri_unstable_times)]
ri_neutral_df = lhflux_measurements_df[lhflux_measurements_df.time.isin(ri_neutral_times)]

tgrad_stable_df = lhflux_measurements_df[lhflux_measurements_df.time.isin(tgrad_stable_times)]
tgrad_unstable_df = lhflux_measurements_df[lhflux_measurements_df.time.isin(tgrad_unstable_times)]
tgrad_neutral_df = lhflux_measurements_df[lhflux_measurements_df.time.isin(tgrad_neutral_times)]

In [117]:
mean_lhflux_profiles = (
    alt.Chart(lhflux_measurements_df).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').title("Mean w'q' (g/m^2/s)").scale(zero=True),
        alt.Y("height:Q")
    ).properties(title=f"All data (n = {len(lhflux_measurements_df)})", width=150, height=150)
 |
    alt.Chart(bs_lhflux_df).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').title("Mean w'q' (g/m^2/s)").scale(zero=True),
        alt.Y("height:Q")
    ).properties(title=f"During blowing snow (n = {len(bs_lhflux_df)})", width=150, height=150)
 |
    alt.Chart(nobs_lhflux_df).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').title("Mean w'q' (g/m^2/s)").scale(zero=True),
        alt.Y("height:Q")
    ).properties(title=f"During no blowing snow (n = {len(nobs_lhflux_df)})", width=150, height=150)
|
    alt.Chart(decoupled_df).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').title("Mean w'q' (g/m^2/s)").scale(zero=True),
        alt.Y("height:Q")
    ).properties(title=f"During decoupling (n = {len(decoupled_df)})", width=150, height=150)
|
    alt.Chart(weaklycoupled_df).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').title("Mean w'q' (g/m^2/s)").scale(zero=True),
        alt.Y("height:Q")
    ).properties(title=f"During weak coupling (n = {len(weaklycoupled_df)})", width=150, height=150)
|
    alt.Chart(coupled_df).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').title("Mean w'q' (g/m^2/s)").scale(zero=True),
        alt.Y("height:Q")
    ).properties(title=f"During coupling (n = {len(coupled_df)})", width=150, height=150)
).configure_point(
    
    size=100
).properties(
    title='Mean latent heat flux profiles during different conditions'
)

mean_lhflux_profiles.resolve_scale(
    y='shared', x='shared'
)

alt.HConcatChart(...)

In [118]:
mean_lhflux_profiles = (
    alt.Chart(lhflux_measurements_df).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').title("Mean w'q' (g/m^2/s)").scale(zero=True),
        alt.Y("height:Q")
    ).properties(title=f"All data (n = {len(lhflux_measurements_df)})", width=150, height=150)
 |
    alt.Chart(bs_lhflux_df).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').title("Mean w'q' (g/m^2/s)").scale(zero=True),
        alt.Y("height:Q")
    ).properties(title=f"During blowing snow (n = {len(bs_lhflux_df)})", width=150, height=150)
 |
    alt.Chart(nobs_lhflux_df).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').title("Mean w'q' (g/m^2/s)").scale(zero=True),
        alt.Y("height:Q")
    ).properties(title=f"During no blowing snow (n = {len(nobs_lhflux_df)})", width=150, height=150)
|
    alt.Chart(ri_stable_df).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').title("Mean w'q' (g/m^2/s)").scale(zero=True),
        alt.Y("height:Q")
    ).properties(title=f"During stable conditions (n = {len(ri_stable_df)})", width=150, height=150)
|
    alt.Chart(ri_neutral_df).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').title("Mean w'q' (g/m^2/s)").scale(zero=True),
        alt.Y("height:Q")
    ).properties(title=f"During neutral conditions (n = {len(ri_neutral_df)})", width=150, height=150)
|
    alt.Chart(ri_unstable_df).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').title("Mean w'q' (g/m^2/s)").scale(zero=True),
        alt.Y("height:Q")
    ).properties(title=f"During unstable conditions (n = {len(ri_unstable_df)})", width=150, height=150)
).configure_point(
    
    size=100
).properties(
    title='Mean latent heat flux profiles during different conditions'
)

mean_lhflux_profiles.resolve_scale(
    y='shared', x='shared'
)

alt.HConcatChart(...)

In [119]:
# all data
################################################################################
all_data_chart = alt.Chart(lhflux_measurements_df).transform_aggregate(
    mean = "mean(value)",
    groupby=['height']
).transform_calculate(
    low_bound = 'datum.mean - 0.2*datum.mean',
    up_bound = 'datum.mean + 0.2*datum.mean'
).mark_errorbar(thickness=2.5).encode(
    alt.X("low_bound:Q").title(""),
    alt.X2("up_bound:Q"),
    alt.Y("height:Q")
).properties(title=f"All data (n = {len(lhflux_measurements_df)})", width=150, height=150) +\
    alt.Chart(lhflux_measurements_df).mark_line(point=True, color='black').encode(
    alt.X("mean(value):Q").sort('-y').title("w'q' (dots) ± 20% (bars) (g/m^2/s)"),
    alt.Y("height:Q")
).properties(title="All data", width=150, height=150)

# blowing snow
################################################################################
bs_chart = alt.Chart(bs_lhflux_df).transform_aggregate(
    mean = "mean(value)",
    groupby=['height']
).transform_calculate(
    low_bound = 'datum.mean - 0.2*datum.mean',
    up_bound = 'datum.mean + 0.2*datum.mean'
).mark_errorbar(thickness=2.5).encode(
    alt.X("low_bound:Q").title(""),
    alt.X2("up_bound:Q"),
    alt.Y("height:Q")
).properties(title=f"BS (n = {len(bs_lhflux_df)})", width=150, height=150) +\
    alt.Chart(bs_lhflux_df).mark_line(point=True, color='black').encode(
    alt.X("mean(value):Q").sort('-y').title("w'q' (dots) ± 20% (bars) (g/m^2/s)"),
    alt.Y("height:Q")
).properties(title="BS", width=150, height=150)

# no blowing snow
################################################################################
nobs_chart = alt.Chart(nobs_lhflux_df).transform_aggregate(
    mean = "mean(value)",
    groupby=['height']
).transform_calculate(
    low_bound = 'datum.mean - 0.2*datum.mean',
    up_bound = 'datum.mean + 0.2*datum.mean'
).mark_errorbar(thickness=2.5).encode(
    alt.X("low_bound:Q").title(""),
    alt.X2("up_bound:Q"),
    alt.Y("height:Q")
).properties(title=f"No BS (n = {len(nobs_lhflux_df)})", width=150, height=150) +\
    alt.Chart(nobs_lhflux_df).mark_line(point=True, color='black').encode(
    alt.X("mean(value):Q").sort('-y').title("w'q' (dots) ± 20% (bars) (g/m^2/s)"),
    alt.Y("height:Q")
).properties(title="No BS", width=150, height=150)


# decoupling
################################################################################
decoupled_chart = alt.Chart(decoupled_df).transform_aggregate(
    mean = "mean(value)",
    groupby=['height']
).transform_calculate(
    low_bound = 'datum.mean - 0.2*datum.mean',
    up_bound = 'datum.mean + 0.2*datum.mean'
).mark_errorbar(thickness=2.5).encode(
    alt.X("low_bound:Q").title(""),
    alt.X2("up_bound:Q"),
    alt.Y("height:Q")
).properties(title=f"Decoupled (n = {len(decoupled_df)})", width=150, height=150) +\
    alt.Chart(decoupled_df).mark_line(point=True, color='black').encode(
    alt.X("mean(value):Q").sort('-y').title("w'q' (dots) ± 20% (bars) (g/m^2/s)"),
    alt.Y("height:Q")
).properties(title="Decoupled", width=150, height=150)


# weaklycoupled_df
################################################################################
weaklycoupled_chart = alt.Chart(weaklycoupled_df).transform_aggregate(
    mean = "mean(value)",
    groupby=['height']
).transform_calculate(
    low_bound = 'datum.mean - 0.2*datum.mean',
    up_bound = 'datum.mean + 0.2*datum.mean'
).mark_errorbar(thickness=2.5).encode(
    alt.X("low_bound:Q").title(""),
    alt.X2("up_bound:Q"),
    alt.Y("height:Q")
).properties(title=f"Weakly coupled (n = {len(weaklycoupled_df)})", width=150, height=150) +\
    alt.Chart(weaklycoupled_df).mark_line(point=True, color='black').encode(
    alt.X("mean(value):Q").sort('-y').title("w'q' (dots) ± 20% (bars) (g/m^2/s)"),
    alt.Y("height:Q")
).properties(title="Weakly coupled", width=150, height=150)

# coupled
################################################################################
coupled_chart = alt.Chart(coupled_df).transform_aggregate(
    mean = "mean(value)",
    groupby=['height']
).transform_calculate(
    low_bound = 'datum.mean - 0.2*datum.mean',
    up_bound = 'datum.mean + 0.2*datum.mean'
).mark_errorbar(thickness=2.5).encode(
    alt.X("low_bound:Q").title(""),
    alt.X2("up_bound:Q"),
    alt.Y("height:Q")
).properties(title=f"Coupled (n = {len(coupled_df)})", width=150, height=150) +\
    alt.Chart(coupled_df).mark_line(point=True, color='black').encode(
    alt.X("mean(value):Q").sort('-y').title("w'q' (dots) ± 20% (bars) (g/m^2/s)"),
    alt.Y("height:Q")
).properties(title="Coupled", width=150, height=150)

# stable
################################################################################
ri_stable_chart = alt.Chart(ri_stable_df).transform_aggregate(
    mean = "mean(value)",
    groupby=['height']
).transform_calculate(
    low_bound = 'datum.mean - 0.2*datum.mean',
    up_bound = 'datum.mean + 0.2*datum.mean'
).mark_errorbar(thickness=2.5).encode(
    alt.X("low_bound:Q").title(""),
    alt.X2("up_bound:Q"),
    alt.Y("height:Q")
).properties(title=f"Ri Stable (n = {len(ri_stable_df)})", width=150, height=150) +\
    alt.Chart(ri_stable_df).mark_line(point=True, color='black').encode(
    alt.X("mean(value):Q").sort('-y').title("w'q' (dots) ± 20% (bars) (g/m^2/s)"),
    alt.Y("height:Q")
).properties(title="Ri Stable", width=150, height=150)

# neutral
################################################################################
ri_neutral_chart = alt.Chart(ri_neutral_df).transform_aggregate(
    mean = "mean(value)",
    groupby=['height']
).transform_calculate(
    low_bound = 'datum.mean - 0.2*datum.mean',
    up_bound = 'datum.mean + 0.2*datum.mean'
).mark_errorbar(thickness=2.5).encode(
    alt.X("low_bound:Q").title(""),
    alt.X2("up_bound:Q"),
    alt.Y("height:Q")
).properties(title=f"Ri Neutral (n = {len(ri_neutral_df)})", width=150, height=150) +\
    alt.Chart(ri_neutral_df).mark_line(point=True, color='black').encode(
    alt.X("mean(value):Q").sort('-y').title("w'q' (dots) ± 20% (bars) (g/m^2/s)"),
    alt.Y("height:Q")
).properties(title="Ri Neutral", width=150, height=150)

# unstable
################################################################################
ri_unstable_chart = alt.Chart(ri_unstable_df).transform_aggregate(
    mean = "mean(value)",
    groupby=['height']
).transform_calculate(
    low_bound = 'datum.mean - 0.2*datum.mean',
    up_bound = 'datum.mean + 0.2*datum.mean'
).mark_errorbar(thickness=2.5).encode(
    alt.X("low_bound:Q").title(""),
    alt.X2("up_bound:Q"),
    alt.Y("height:Q")
).properties(title=f"Ri Unstable (n = {len(ri_unstable_df)})", width=150, height=150) +\
    alt.Chart(ri_unstable_df).mark_line(point=True, color='black').encode(
    alt.X("mean(value):Q").sort('-y').title("w'q' (dots) ± 20% (bars) (g/m^2/s)"),
    alt.Y("height:Q")
).properties(title="Ri Unstable", width=150, height=150)


# tgrad_stable_df
################################################################################
tgrad_stable_chart = alt.Chart(tgrad_stable_df).transform_aggregate(
    mean = "mean(value)",
    groupby=['height']
).transform_calculate(
    low_bound = 'datum.mean - 0.2*datum.mean',
    up_bound = 'datum.mean + 0.2*datum.mean'
).mark_errorbar(thickness=2.5).encode(
    alt.X("low_bound:Q").title(""),
    alt.X2("up_bound:Q"),
    alt.Y("height:Q")
).properties(title=f"Tgrad stable (n = {len(tgrad_stable_df)})", width=150, height=150) +\
    alt.Chart(tgrad_stable_df).mark_line(point=True, color='black').encode(
    alt.X("mean(value):Q").sort('-y').title("w'q' (dots) ± 20% (bars) (g/m^2/s)"),
    alt.Y("height:Q")
).properties(title="Tgrad stable", width=150, height=150)


# tgrad_neutral_df
################################################################################
tgrad_neutral_chart = alt.Chart(tgrad_neutral_df).transform_aggregate(
    mean = "mean(value)",
    groupby=['height']
).transform_calculate(
    low_bound = 'datum.mean - 0.2*datum.mean',
    up_bound = 'datum.mean + 0.2*datum.mean'
).mark_errorbar(thickness=2.5).encode(
    alt.X("low_bound:Q").title(""),
    alt.X2("up_bound:Q"),
    alt.Y("height:Q")
).properties(title=f"Tgrad Neutral (n = {len(tgrad_neutral_df)})", width=150, height=150) +\
    alt.Chart(tgrad_neutral_df).mark_line(point=True, color='black').encode(
    alt.X("mean(value):Q").sort('-y').title("w'q' (dots) ± 20% (bars) (g/m^2/s)"),
    alt.Y("height:Q")
).properties(title="Tgrad Neutral", width=150, height=150)

# tgrad_unstable_df
################################################################################
tgrad_unstable_chart = alt.Chart(tgrad_unstable_df).transform_aggregate(
    mean = "mean(value)",
    groupby=['height']
).transform_calculate(
    low_bound = 'datum.mean - 0.2*datum.mean',
    up_bound = 'datum.mean + 0.2*datum.mean'
).mark_errorbar(thickness=2.5).encode(
    alt.X("low_bound:Q").title(""),
    alt.X2("up_bound:Q"),
    alt.Y("height:Q")
).properties(title=f"Tgrad Unstable (n = {len(tgrad_unstable_df)})", width=150, height=150) +\
    alt.Chart(tgrad_unstable_df).mark_line(point=True, color='black').encode(
    alt.X("mean(value):Q").sort('-y').title("w'q' (dots) ± 20% (bars) (g/m^2/s)"),
    alt.Y("height:Q")
).properties(title="Tgrad Unstable", width=150, height=150)





(
    (all_data_chart | bs_chart | nobs_chart).resolve_scale(x='shared')
    &
    (decoupled_chart | weaklycoupled_chart | coupled_chart).resolve_scale(x='shared')
    &
    (ri_stable_chart | ri_neutral_chart | ri_unstable_chart).resolve_scale(x='shared')
    & 
    (tgrad_stable_chart |  tgrad_neutral_chart | tgrad_unstable_chart).resolve_scale(x='shared')
).configure_point(
    size=50, color='black'
).resolve_scale(y='shared', x='shared').properties(title="Mean w'q' at each measurement, showing +/- 20%")





alt.VConcatChart(...)

In [120]:
bs_and_coupled_df = lhflux_measurements_df[lhflux_measurements_df.time.isin(
    set(bs_times).union(set(coupled_times)) 
)]
nobs_and_coupled_df = lhflux_measurements_df[lhflux_measurements_df.time.isin(
    set(nobs_times).union(set(coupled_times)) 
)]

# during_bs_and_coupling
################################################################################
during_bs_and_coupling = alt.Chart(bs_and_coupled_df).transform_aggregate(
    median = "median(value)",
    groupby=['height']
).transform_calculate(
    low_bound = 'datum.median - 0.2*datum.median',
    up_bound = 'datum.median + 0.2*datum.median'
).mark_errorbar(thickness=2.5).encode(
    alt.X("low_bound:Q").title(""),
    alt.X2("up_bound:Q"),
    alt.Y("height:Q")
).properties(title="During BS and coupling", width=150, height=150) +\
    alt.Chart(bs_and_coupled_df).mark_line(point=True, color='black').encode(
    alt.X("median(value):Q").sort('-y').title("w'q' (dots) ± 20% (bars) (g/m^2/s)"),
    alt.Y("height:Q")
).properties(title="During BS and coupling", width=150, height=150)

# nobs_and_coupling
################################################################################
nobs_and_coupling = alt.Chart(nobs_and_coupled_df).transform_aggregate(
    median = "median(value)",
    groupby=['height']
).transform_calculate(
    low_bound = 'datum.median - 0.2*datum.median',
    up_bound = 'datum.median + 0.2*datum.median'
).mark_errorbar(thickness=2.5).encode(
    alt.X("low_bound:Q").title(""),
    alt.X2("up_bound:Q"),
    alt.Y("height:Q")
).properties(title="During no BS and coupling", width=150, height=150) +\
    alt.Chart(nobs_and_coupled_df).mark_line(point=True, color='black').encode(
    alt.X("median(value):Q").sort('-y').title("w'q' (dots) ± 20% (bars) (g/m^2/s)"),
    alt.Y("height:Q")
).properties(title="During BS and coupling", width=150, height=150)

(during_bs_and_coupling | nobs_and_coupling).configure_point(
    size=50, color='black'
).resolve_scale(y='shared', x='independent').properties(title="Mean w'q' at each measurement, showing +/- 20%")

alt.HConcatChart(...)

## Plot vertical profiles exploring the December wind event

Sensible heat

In [121]:
src = tidy_df[tidy_df.variable.isin([
    'w_tc__1m_c', 'w_tc__2m_c', 'w_tc__3m_c', 'w_tc__5m_c', 'w_tc__10m_c', 'w_tc__15m_c', 'w_tc__20m_c',
    'w_tc__1m_ue',  'w_tc__1m_uw',  'w_tc__1m_d',
    'w_tc__3m_ue',  'w_tc__3m_uw',  'w_tc__3m_d',
    'w_tc__10m_ue', 'w_tc__10m_uw', 'w_tc__10m_d',
])]
# Tower uw was buried during this event, so lets not look at that data
src = src.query("tower != 'uw'")
w_tc_prof_timeseries_chart = alt.Chart(
    src[
        src.time.dt.minute == 0
    ].set_index('time').loc[
        "2022-12-21 12": "2022-12-21 23"
    ].reset_index()
).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').title("w'tc'"), #.scale(domain=[-0.25,0], clamp=True),
        alt.Y("height:Q"),
        alt.Color("tower:N"),
        # alt.Color("hours(time):O"),
        alt.Facet("time:T").header(labelFontSize=0).title(None),
    ).properties(width=75,height=75)

Potential temperature

In [23]:
src = tidy_df[tidy_df.measurement.isin([
    'surface potential temperature',
    'potential temperature'
])].query("tower == 'c'")
theta_prof_timeseries_chart = alt.Chart(
    src[
        src.time.dt.minute == 0
    ].set_index('time').loc[
        "2022-12-21 12": "2022-12-21 23"
    ].reset_index()
).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').title("theta").scale(zero=False),
        alt.Y("height:Q"),
        # alt.Color("hours(time):O"),
        alt.Facet("time:T").header(labelFontSize=0).title(None),
        tooltip='height:Q'
    ).properties(width=75,height=75)

Temperature

In [24]:
src = tidy_df[tidy_df.measurement.isin([
    'surface temperature',
    'temperature'
])].query("tower == 'c'")
T_prof_timeseries_chart = alt.Chart(
    src[
        src.time.dt.minute == 0
    ].set_index('time').loc[
        "2022-12-21 12": "2022-12-21 23"
    ].reset_index()
).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').title("theta").scale(zero=False),
        alt.Y("height:Q"),
        # alt.Color("hours(time):O"),
        alt.Facet("time:T").header(labelFontSize=0).title(None),
        tooltip='height:Q'
    ).properties(width=75,height=75)
T_prof_timeseries_chart

alt.Chart(...)

lh flux 

In [25]:
src = tidy_df[tidy_df.variable.isin([
    'w_h2o__1m_c', 'w_h2o__2m_c', 'w_h2o__3m_c', 'w_h2o__5m_c', 'w_h2o__10m_c', 'w_h2o__15m_c', 'w_h2o__20m_c',
    'w_h2o__1m_ue',  'w_h2o__1m_uw',  'w_h2o__1m_d',
    'w_h2o__3m_ue',  'w_h2o__3m_uw',  'w_h2o__3m_d',
    'w_h2o__10m_ue', 'w_h2o__10m_uw', 'w_h2o__10m_d',
])]
# Tower uw was buried during this event, so lets not look at that data
src = src.query("tower != 'uw'")
w_q_prof_timeseries_chart = alt.Chart(
    src[
        src.time.dt.minute == 0
    ].set_index('time').loc[
        "2022-12-21 12": "2022-12-21 23"
    ].reset_index()
).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').scale(domain=[0, 0.05], clamp=True).title("w'q'"),
        alt.Y("height:Q"),
        alt.Color("tower:N"),
        # alt.Color("hours(time):O"),
        alt.Facet("time:T").header(format="%m/%d %H:%M")
    ).properties(width=75,height=75)

RH

In [26]:
src = tidy_df[tidy_df.measurement=='RH'].query("height != 6").query("height != 12")
rh_prof_timeseries_chart = alt.Chart(
    src[
        src.time.dt.minute == 0
    ].set_index('time').loc[
        "2022-12-21 12": "2022-12-21 23"
    ].reset_index()
).mark_circle().encode(
        alt.X("mean(value):Q").sort('-y').title("RH (%)").scale(domain=[55,75]),
        alt.Y("height:Q"),
        alt.Color("tower:N"),
        # alt.Color("hours(time):O"),
        alt.Facet("time:T").header(labelFontSize=0).title(None),
        tooltip='height:Q'
    ).properties(width=75,height=75)

blowing snow time series

In [27]:
# src = tidy_df[tidy_df.variable == 'SF_avg_ue']
src = tidy_df[tidy_df.variable.isin(['SF_avg_1m_ue', 'SF_avg_2m_ue'])]
# src = src[src.time.isin(bs_times)]
bs_flux_timeseries_chart = alt.Chart(
    src.set_index('time').loc[
        "2022-12-21 12": "2022-12-21 23"
    ].reset_index().query("value > 0")
).mark_circle(point=True, size=75).encode(
        alt.X("time:T"),
        alt.Y("value:Q").scale(type='log').title("Blowing snow flux (g/m^2/s)"),
        alt.Color("height:O")
).properties(width=1140,height=100) 

src = tidy_df[tidy_df.variable.isin(['spd_3m_c', 'spd_5m_c', 'spd_20m_c'])]
windspd_timeseries_chart = alt.Chart(
    src.set_index('time').loc[
        "2022-12-21 12": "2022-12-21 23"
    ].reset_index().query("value > 0")
).mark_line().encode(
        alt.X("time:T").axis(labels=False).title(None),
        alt.Y("value:Q").title("Wind speed (m/s)"),
        alt.Color("height:O")
).properties(width=1140,height=100) 


src = tidy_df[tidy_df.variable.isin(['u*_3m_c', 'u*_5m_c', 'u*_20m_c'])]
ufric_timeseries_chart = alt.Chart(
    src.set_index('time').loc[
        "2022-12-21 12": "2022-12-21 23"
    ].reset_index().query("value > 0")
).mark_line().encode(
        alt.X("time:T").axis(labels=False).title(None),
        alt.Y("value:Q").title("Friction velocity (m/s)"),
        alt.Color("height:O")
).properties(width=1140,height=100) 


In [28]:
(
    ufric_timeseries_chart & windspd_timeseries_chart & bs_flux_timeseries_chart & 
    (w_q_prof_timeseries_chart & rh_prof_timeseries_chart & w_tc_prof_timeseries_chart & theta_prof_timeseries_chart)
).resolve_scale(color='independent')

alt.VConcatChart(...)

Look at time series of Snow surface temperature, and other related variables, to see what is going on with surface temperatures during the blowing snow event

In [29]:
src = tidy_df.set_index('time').sort_index().loc[
    "2022-12-21 12": "2022-12-22 23"
].reset_index()
# Create DF with renamed Tsnow variables to clarify that they are air sensors at this time because
# snowdepth during this time period is 30-40cm, so the in-snow T sensors are ~10, 20cm above snow
in_snow_sensors_as_air_sensors_df = src[src.variable.isin([
    'T_2m_c',
    'Tsnow_0_4m_d', 'Tsnow_0_5m_d', 'Tsnow_0_6m_d', 'Tsnow_0_7m_d',
    'SnowDepth_d'
])].pivot(index='time', columns='variable', values='value')
in_snow_sensors_as_air_sensors_df['T_00cm_c'] = in_snow_sensors_as_air_sensors_df['Tsnow_0_4m_d']
in_snow_sensors_as_air_sensors_df['T_10cm_c'] = in_snow_sensors_as_air_sensors_df['Tsnow_0_5m_d']
in_snow_sensors_as_air_sensors_df['T_20cm_c'] = in_snow_sensors_as_air_sensors_df['Tsnow_0_6m_d']
in_snow_sensors_as_air_sensors_df['T_30cm_c'] = in_snow_sensors_as_air_sensors_df['Tsnow_0_7m_d']
in_snow_sensors_as_air_sensors_df = in_snow_sensors_as_air_sensors_df.reset_index().melt(id_vars='time')   

# Create DF to hold the COARE model results we care about here
# and to rename the run we did using the in-snow temp
coare_model_results_local = coare_model_results[coare_model_results.config.isin([
    "Tsurf_d e_sat_alduchov 0.0005",
    "Tsurf_c e_sat_alduchov 0.0005",
    "Tsnow_0_4m_d e_sat_alduchov 0.0005",
])].set_index("time").sort_index().loc[
    src.time.min():src.time.max()
].reset_index()
coare_model_results_local.surface_measurement = coare_model_results_local.surface_measurement.replace(
    "Tsnow_0_4m_d",
    'T_00cm_c'
)

bs_case_study =(alt.Chart(src[src.variable == 'SF_avg_ue']).mark_line().encode(
    alt.X("time:T").axis(labels=False).title(None),
    alt.Y("value:Q").title("BS flux (g/m^2/s)"),
).properties(height = 100) &\
alt.Chart(in_snow_sensors_as_air_sensors_df[in_snow_sensors_as_air_sensors_df.variable.isin(
    ['T_00cm_c', 'T_10cm_c', 'T_20cm_c', 'T_30cm_c', 'T_2m_c']
)]).mark_line().encode(
    alt.X("time:T").axis(labels=False).title(None),
    alt.Y("value:Q").title("Temp. (˚C)"),
    alt.Color("variable:O").scale(
        domain = ['T_00cm_c', 'T_10cm_c', 'T_20cm_c', 'T_30cm_c', 'T_2m_c'],
        range=['lightsteelblue', 'cornflowerblue', 'royalblue', 'blue', 'black']
    ).title(["Temp. Measurement", "(near-surface and air)"]),
).resolve_scale(x='shared', y='independent', color='independent').properties(height = 100) &\
alt.Chart(src[src.variable.isin([
    'T_2m_c',
    'Tsurf_d',
    'Tsurf_c',
    'Tsurf_uw',
    'Tsurf_ue',
    'Tsurf_rad_d',
])]).mark_line().encode(
    alt.X("time:T").axis(labels=False).title(None),
    alt.Y("value:Q").title("Temp. (˚C)"),
    alt.Color("variable:O").scale(range=[
        'black', '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd'
    ]).title(["Temp. Measurement", "(surface and air)"]),
    detail='variable'
).resolve_scale(x='shared', y='independent').properties(height = 100)).resolve_scale(color='independent', strokeDash='independent') &\
alt.Chart(
    tidy_df.query("variable == 'w_h2o__3m_c'").set_index("time").loc[
        "20221221": "20221223"
    ].reset_index()
).mark_line(color='black').encode(
    alt.X("time:T"),
    alt.Y("value:Q"),
).properties(height = 100) + alt.Chart(
    coare_model_results_local
).mark_line().encode(
    alt.X("time:T"),
    alt.Y("hlb_gperm2s").title("w'q' (g/m^s/2)"),
    alt.Color("surface_measurement").title([
        "Tsurf used for",
        "model input"
    ])
)

In [30]:
src = tidy_df.set_index('time').sort_index().loc[
    "20230210": "20230212"
].reset_index()
# rename the Tsnow variables so clarify that the snowdepth during this time period is 88cm, 
# so the in-snow T sensors are actually ~10, 20, cm above the snow surface
# When we looked at T sensors at 80cm and 90cm, they were clearly under snow (signal attenuated)
# So we think its a decent estimate that the snow by the instrument is actually ~1m deep
in_snow_sensors_as_air_sensors_df = src[src.variable.isin([
    'T_2m_c',
    'Tsnow_1_0m_d',
    'Tsnow_1_1m_d',
    'Tsnow_1_2m_d',
    'Tsnow_1_3m_d',
    # 'Tsnow_1_4m_d',
    'SnowDepth_d'
])].pivot(index='time', columns='variable', values='value')
in_snow_sensors_as_air_sensors_df['T_00cm_c'] = in_snow_sensors_as_air_sensors_df['Tsnow_1_0m_d']
in_snow_sensors_as_air_sensors_df['T_10cm_c'] = in_snow_sensors_as_air_sensors_df['Tsnow_1_1m_d']
in_snow_sensors_as_air_sensors_df['T_20cm_c'] = in_snow_sensors_as_air_sensors_df['Tsnow_1_2m_d']
in_snow_sensors_as_air_sensors_df['T_30cm_c'] = in_snow_sensors_as_air_sensors_df['Tsnow_1_3m_d']
in_snow_sensors_as_air_sensors_df = in_snow_sensors_as_air_sensors_df.reset_index().melt(id_vars='time')

# Create DF to hold the COARE model results we care about here
# and to rename the run we did using the in-snow temp
coare_model_results_local = coare_model_results[coare_model_results.config.isin([
    "Tsurf_d e_sat_alduchov 0.0005",
    "Tsurf_c e_sat_alduchov 0.0005",
    "Tsnow_1_0m_d e_sat_alduchov 0.0005",
])].set_index("time").sort_index().loc[
    src.time.min():src.time.max()
].reset_index()
coare_model_results_local.surface_measurement = coare_model_results_local.surface_measurement.replace(
    'Tsnow_1_0m_d',
    'T_00cm_c'
)

clear_case_study = (alt.Chart(src[src.variable == 'SF_avg_ue']).mark_line().encode(
    alt.X("time:T").axis(labels=False).title(None),
    alt.Y("value:Q").title("BS flux (g/m^2/s)"),
).properties(height = 100) &\
alt.Chart(in_snow_sensors_as_air_sensors_df[in_snow_sensors_as_air_sensors_df.variable.isin(
    ['T_2m_c', 'T_00cm_c', 'T_10cm_c', 'T_20cm_c','T_30cm_c']
)]).mark_line().encode(
    alt.X("time:T").axis(labels=False).title(None),
    alt.Y("value:Q").title("Temp. (˚C)"),
    alt.Color("variable:O").scale(
        domain = ['T_00cm_c', 'T_10cm_c', 'T_20cm_c','T_30cm_c', 'T_2m_c', ],
        range=['lightsteelblue', 'cornflowerblue', 'royalblue', 'blue', 'black']
    ).title(["Temp. Measurement", "(near-surface and air)"]),
    # alt.StrokeDash("measurement:N"),
    detail='variable'
).resolve_scale(x='shared', y='independent', color='independent').properties(height = 100) &\
alt.Chart(src[src.variable.isin([
    'T_2m_c',
    'Tsurf_d',
    'Tsurf_c',
    'Tsurf_uw',
    'Tsurf_ue',
    'Tsurf_rad_d',
])]).mark_line().encode(
    alt.X("time:T").axis(labels=False).title(None),
    alt.Y("value:Q").title("Temp. (˚C)"),
    alt.Color("variable:O").scale(range=[
        'black', '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd'
    ]).title(["Temp. Measurement", "(surface and air)"]),
    # alt.StrokeDash("measurement:N"),
    detail='variable'
).resolve_scale(x='shared', y='independent').properties(height = 100)).resolve_scale(color='independent', strokeDash='independent') &\
alt.Chart(
    tidy_df.query("variable == 'w_h2o__3m_c'").set_index("time").loc[
        "20230210": "20230212"
    ].reset_index()
).mark_line(color='black').encode(
    alt.X("time:T"),
    alt.Y("value:Q")
).properties(height = 100) + alt.Chart(
    coare_model_results_local
).mark_line().encode(
    alt.X("time:T"),
    alt.Y("hlb_gperm2s").title("w'q' (g/m^s/2)"),
    alt.Color("surface_measurement:N").title([
        "Tsurf used for",
        "model input"
    ])
)

In [31]:
(bs_case_study | clear_case_study)

alt.HConcatChart(...)

# Examine how divergence varies with distance from snow surface

In [61]:
div_variation = tidy_df[tidy_df.variable.isin([
    # 'w_h2o__1m_c',
    'w_h2o__2m_c',
    'w_h2o__3m_c',
    'w_h2o__5m_c',
    'w_h2o__10m_c',
    'w_h2o__15m_c',
    'w_h2o__20m_c',
    'SnowDepth_d',
])].pivot(index='time', columns='variable', values='value').reset_index()
div_variation['height_adj'] = 2 - div_variation['SnowDepth_d']

In [62]:
div_variation['variation_2_3'] = (div_variation['w_h2o__3m_c'] - div_variation['w_h2o__2m_c']) / div_variation[['w_h2o__2m_c', 'w_h2o__3m_c']].mean(axis=1)

In [63]:
div_variation.variation_2_3.mean(), div_variation.variation_2_3.median(), div_variation.variation_2_3.max(), div_variation.variation_2_3.min()

(0.05255324798085847,
 0.026466484698459806,
 1131.0214224507283,
 -954.5545722713864)

In [64]:
alt.Chart(div_variation).transform_filter(
    alt.datum.variation_2_3 <= 1
).transform_filter(
    alt.datum.variation_2_3 >= -1
).mark_boxplot().encode(
    alt.X("height_adj:Q").bin(True),
    alt.Y("variation_2_3:Q")
)

alt.Chart(...)